In [ ]:
# model: https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
# model: https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
# predict ahead: https://towardsdatascience.com/time-series-forecasting-with-recurrent-neural-networks-74674e289816

In [46]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os, math
import preprocess

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Masking

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [322]:
data = pd.read_csv("Data/Electrical_Averaged.csv")
data["Timestamp"] = pd.to_datetime(data['Timestamp'])
data.set_index('Timestamp', inplace=True)
building_data = preprocess.remove_outliers(data, 15)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(building_data)

n_steps_in = 30
n_steps_out = 1
mask_value = 0
tf.random.set_seed(1)

# set check point
checkpoint_path = f"training_1/checkpoint__{n_steps_in}in_{n_steps_out}_out"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

def generate_sequence(data, n_steps_in, n_steps_out):
    X, y = list(), list()
    for index in range(n_steps_in, len(data)-n_steps_out+1):
        X.append(data[index-n_steps_in : index])
        y.append(data[index : index+n_steps_out])
    return np.array(X), np.array(y)

scaled_data[np.isnan(scaled_data)] = mask_value
X, y = generate_sequence(scaled_data, n_steps_in, n_steps_out)

n_features = X.shape[2]
print(scaled_data.shape, X.shape, y.shape)

# define model
model = Sequential()
model.add(Masking(mask_value= mask_value, input_shape=(n_steps_in, n_features)))
model.add(LSTM(200, activation='relu'))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')

(3661, 12) (3631, 30, 12) (3631, 1, 12)


In [364]:
# Restore the weights
model.load_weights(f'./Weight/weight_{n_steps_in}in_{n_steps_out}_out')

ahead = 365

def predict_ahead(X, model, n_ahead):
    yhat = list()
    X_copy = np.copy(X)
    for _ in range(n_ahead):
        prediction = model.predict(X_copy, verbose=0)
        yhat.append(prediction[0, 0])
        X_copy = np.concatenate((X_copy, prediction), axis=1)[:,1:]
    return np.array(yhat)
yhat = predict_ahead(X[-1:], model, ahead)    
yhat = scaler.inverse_transform(yhat)
time_period = pd.date_range(building_data.index[-1], periods=ahead+1, freq="d")[1:]
yhat = pd.DataFrame(yhat, columns = building_data.columns, index = time_period)
yhat.index.name = "Timestamp"

In [369]:
yhat.index.name = "Timestamp"
yhat.to_csv("Data/Electrical_Predict.csv")

In [349]:
yhat.reset_index(drop=True, inplace=True)

In [344]:
yhat = pd.DataFrame(yhat, columns = building_data.columns)

In [ ]:
df.append(data_to_append, ignore_index = True)

In [ ]:
# fit model
model.fit(X, y, epochs=200, verbose=0, callbacks=[cp_callback])
# Save the weights
model.save_weights(f'./Weight/weight_{n_steps_in}in_{n_steps_out}_out')

In [ ]:
# fit model
model.fit(X, y, epochs=200, verbose=0, callbacks=[cp_callback])
# Save the weights
model.save_weights(f'./Weight/weight_{n_steps_in}in_{n_steps_out}_out')

In [ ]:
# # split a multivariate sequence into samples
# def split_sequences(sequences, n_steps_in, n_steps_out):
#     X, y = list(), list()
#     for i in range(len(sequences)):
#         # find the end of this pattern
#         end_ix = i + n_steps_in
#         out_end_ix = end_ix + n_steps_out
#         # check if we are beyond the dataset
#         if out_end_ix > len(sequences):
#             break
#         # gather input and output parts of the pattern
#         seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
#         X.append(seq_x)
#         y.append(seq_y)
#     return array(X), array(y)
 
# # define input sequence
# in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
# in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
# out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# # convert to [rows, columns] structure
# in_seq1 = in_seq1.reshape((len(in_seq1), 1))
# in_seq2 = in_seq2.reshape((len(in_seq2), 1))
# out_seq = out_seq.reshape((len(out_seq), 1))
# # horizontally stack columns
# dataset = hstack((in_seq1, in_seq2, out_seq))
# # choose a number of time steps
# n_steps_in, n_steps_out = 3, 2
# # covert into input/output
# X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# # the dataset knows the number of features, e.g. 2
# n_features = X.shape[2]
# print(X.shape, y.shape)
# # summarize the data
# for i in range(len(X)):
#     print(X[i], y[i])